<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
import pandas as pd
from janitor import clean_names

In [2]:
# Reading the json as a dict
with open('data/scraped_fines.json') as json_data:
    fines = json.load(json_data)

In [3]:
fines_df = (
    pd
    .json_normalize(fines['allItems'])
    .rename(columns ={'articleViolated':'article_violated'})
)

In [4]:
fines_df

,id,picture,name,price,authority,date,controller,article_violated,type,source,summary
0,1,https://www.privacyaffairs.com/wp-content/uplo...,Poland,9380,Polish National Personal Data Protection Offic...,10/18/2019,Polish Mayor,Art. 28 GDPR,Non-compliance with lawful basis for data proc...,https://uodo.gov.pl/decyzje/ZSPU.421.3.2019,No data processing agreement has been conclude...
1,2,https://www.privacyaffairs.com/wp-content/uplo...,Romania,2500,Romanian National Supervisory Authority for Pe...,10/17/2019,UTTIS INDUSTRIES,"Art. 12 GDPR, Art. 13 GDPR, Art. 5 (1) c) GDPR...",Information obligation non-compliance,https://www.dataprotection.ro/?page=A_patra_am...,A controller was sanctioned because he had unl...
2,3,https://www.privacyaffairs.com/wp-content/uplo...,Spain,60000,Spanish Data Protection Authority (AEPD),10/16/2019,Xfera Moviles S.A.,"Art. 5 GDPR, Art. 6 GDPR",Non-compliance with lawful basis for data proc...,https://www.aepd.es/resoluciones/PS-00262-2019...,The company had unlawfully processed the perso...
3,4,https://www.privacyaffairs.com/wp-content/uplo...,Spain,8000,Spanish Data Protection Authority (AEPD),10/16/2019,Iberdrola Clientes,Art. 31 GDPR,Failure to cooperate with supervisory authority,https://www.aepd.es/resoluciones/PS-00304-2019...,Iberdrola Clientes violated Article 13 of the ...
4,5,https://www.privacyaffairs.com/wp-content/uplo...,Romania,150000,Romanian National Supervisory Authority for Pe...,10/09/2019,Raiffeisen Bank SA,Art. 32 GDPR,Failure to implement sufficient measures to en...,https://www.dataprotection.ro/?page=Comunicat_...,Raiffeisen Bank Romania did not observe the ne...
...,...,...,...,...,...,...,...,...,...,...,...
875,876,https://www.privacyaffairs.com/wp-content/uplo...,Spain,56000,Spanish Data Protection Authority (AEPD),09/14/2021,"Vodafone Espana, S.A.U.",Art. 6 (1) GDPR,Non-compliance with lawful basis for data proc...,,The Spanish DPNA (AEPD) has fined Vodafone Esp...
876,877,https://www.privacyaffairs.com/wp-content/uplo...,Spain,56000,Spanish Data Protection Authority (AEPD),09/14/2021,"Vodafone Espana, S.A.U.",Art. 6 (1) GDPR,Non-compliance with lawful basis for data proc...,,The Spanish DPA (also known as AEPD) fined Vo...
877,878,https://www.privacyaffairs.com/wp-content/uplo...,Spain,4000,Spanish Data Protection Authority (AEPD),09/16/2021,Frigorifica Botana S.L.,Art. 5 (1) c) GDPR,Failure to comply with data processing principles,,The Spanish DPA (also known as AEPD) has fined...
878,879,https://www.privacyaffairs.com/wp-content/uplo...,Spain,18000,Spanish Data Protection Authority (AEPD),09/20/2021,CEDICO,Art. 5 (1) f) GDPR,Failure to comply with data processing principles,,"The Spanish DPA (AEPD) has fined CEDICO, CENTR..."


In [5]:
with open('data/scraped_art_text.json') as json_data:
    art_text = json.load(json_data)

In [107]:
art_text = (
    pd
    .read_json('data/scraped_art_text.json')
    .T
    .reset_index()
    .rename({'index':'chapter', 'title':'chapter_title', 'link':'chapter_link'}
            ,axis='columns')
    .assign(chapter=lambda x: x.chapter.str.extract(r'(\d+)').astype('int'))
)

In [109]:
# empty df, loop will add rows to the bottom
long = pd.DataFrame()

for i in range(len(art_text.chapter)):
    unnested_articles = (
        pd
        .DataFrame(art_text.articles[i])
        .T
        .assign(chapter=i + 1)
        
    )
    # finally add this to initialised df
    long=pd.concat([long, unnested_articles], axis='rows')



In [113]:
art_text.merge(long, how='left', on='chapter').drop(columns='articles').text.str.split('\n')

0     [, This Regulation lays down rules relating to...
1     [, This Regulation applies to the processing o...
2     [, This Regulation applies to the processing o...
3     [, ‘personal data’ means any information relat...
4     [, Personal data shall be:, , processed lawful...
                            ...                        
94    [This Regulation shall not impose additional o...
95    [International agreements involving the transf...
96    [, 1By 25 May 2020 and every four years therea...
97    [1The Commission shall, if appropriate, submit...
98    [, This Regulation shall enter into force on t...
Name: text, Length: 99, dtype: object

In [97]:
art_text.merge(unnested_articles, how='left', on='chapter')

,chapter,title,link,articles,art_title,art_link,art_text
0,1,General provisions,http://gdpr-info.eu/chapter-1/,{'1': {'title': 'Subject-matter and objectives...,NaN,NaN,NaN
1,2,Principles,http://gdpr-info.eu/chapter-2/,{'5': {'title': 'Principles relating to proces...,NaN,NaN,NaN
2,3,Rights of the data subject,http://gdpr-info.eu/chapter-3/,"{'12': {'title': 'Transparent information, com...",NaN,NaN,NaN
3,4,Controller and processor,http://gdpr-info.eu/chapter-4/,{'24': {'title': 'Responsibility of the contro...,NaN,NaN,NaN
4,5,Transfers of personal data to third countries ...,http://gdpr-info.eu/chapter-5/,{'44': {'title': 'General principle for transf...,NaN,NaN,NaN
5,6,Independent supervisory authorities,http://gdpr-info.eu/chapter-6/,"{'51': {'title': 'Supervisory authority', 'lin...",NaN,NaN,NaN
6,7,Cooperation and consistency,http://gdpr-info.eu/chapter-7/,{'60': {'title': 'Cooperation between the lead...,NaN,NaN,NaN
7,8,"Remedies, liability and penalties",http://gdpr-info.eu/chapter-8/,{'77': {'title': 'Right to lodge a complaint w...,NaN,NaN,NaN
8,9,Provisions relating to specific processing sit...,http://gdpr-info.eu/chapter-9/,{'85': {'title': 'Processing and freedom of ex...,NaN,NaN,NaN
9,10,Delegated acts and implementing acts,http://gdpr-info.eu/chapter-10/,"{'92': {'title': 'Committee procedure', 'link'...",NaN,NaN,NaN


In [28]:
art_text.reset_index()

,index,title,link,articles
0,Chapter 1,General provisions,http://gdpr-info.eu/chapter-1/,{'1': {'title': 'Subject-matter and objectives...
1,Chapter 2,Principles,http://gdpr-info.eu/chapter-2/,{'5': {'title': 'Principles relating to proces...
2,Chapter 3,Rights of the data subject,http://gdpr-info.eu/chapter-3/,"{'12': {'title': 'Transparent information, com..."
3,Chapter 4,Controller and processor,http://gdpr-info.eu/chapter-4/,{'24': {'title': 'Responsibility of the contro...
4,Chapter 5,Transfers of personal data to third countries ...,http://gdpr-info.eu/chapter-5/,{'44': {'title': 'General principle for transf...
5,Chapter 6,Independent supervisory authorities,http://gdpr-info.eu/chapter-6/,"{'51': {'title': 'Supervisory authority', 'lin..."
6,Chapter 7,Cooperation and consistency,http://gdpr-info.eu/chapter-7/,{'60': {'title': 'Cooperation between the lead...
7,Chapter 8,"Remedies, liability and penalties",http://gdpr-info.eu/chapter-8/,{'77': {'title': 'Right to lodge a complaint w...
8,Chapter 9,Provisions relating to specific processing sit...,http://gdpr-info.eu/chapter-9/,{'85': {'title': 'Processing and freedom of ex...
9,Chapter 10,Delegated acts and implementing acts,http://gdpr-info.eu/chapter-10/,"{'92': {'title': 'Committee procedure', 'link'..."
